## Data Cleaning

In [4]:
pip install tqdm


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install emot


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install gensim


Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install language-tool-python


Note: you may need to restart the kernel to use updated packages.


In [9]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.1 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
# pip install textblob
pip install tqdm
pip install emot
pip install spacy
pip install gensim
pip install language-tool-python
# !python -m spacy download en
!python -m spacy download en_core_web_sm

In [10]:
from tqdm import tqdm
import pandas as pd
import spacy

In [173]:
df = pd.read_csv('/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/concatenated_results1.csv')

/var/folders/h2/q_r6y6bd5f5c9hvfsfnt31xr0000gn/T/ipykernel_10037/991846668.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/concatenated_results1.csv')


In [174]:
df = df.drop(columns = 'Unnamed: 0')

In [122]:
# Remove duplicates based on the 'text' column
df = df.drop_duplicates(subset='text', keep='first')
# df = df.drop_duplicates(inplace=True)

df
label_counts = Counter(df['Cyberbullying'])
print("Label counts after remove duplicates", label_counts)

Label counts after remove duplicates Counter({0: 91836, 1: 5154})


In [13]:
def get_vocab(corpus):
    '''
    Function returns unique words in document corpus
    '''
    # vocab set
    unique_words = set()
    
    # looping through each document in corpus
    for document in tqdm(corpus):
        for word in document.split(" "):
            if len(word) > 2:
                unique_words.add(word)
    
    return unique_words

def create_profane_mapping(profane_words,vocabulary):
    '''
    Function creates a mapping between commonly found profane words and words in 
    document corpus 
    '''
    
    # mapping dictionary
    mapping_dict = dict()
    
    # looping through each profane word
    for profane in tqdm(profane_words):
        mapped_words = set()
        
        # looping through each word in vocab
        for word in vocabulary:
            # mapping only if ratio > 80
            try:
                if fuzz.ratio(profane,word) > 90:
                    mapped_words.add(word)
            except:
                pass
                
        # list of all vocab words for given profane word
        mapping_dict[profane] = mapped_words
    
    return mapping_dict
def replace_words(corpus,mapping_dict):
    '''
    Function replaces obfuscated profane words using a mapping dictionary
    '''
    
    processed_corpus = []
    
    # iterating over each document in the corpus
    for document in tqdm(corpus):
        
        # splitting sentence to word
        comment = document.split()
        
        # iterating over mapping_dict
        for mapped_word,v in mapping_dict.items():
            
            # comparing target word to each comment word 
            for target_word in v:
                
                # each word in comment
                for i,word in enumerate(comment):
                    if word == target_word:
                        comment[i] = mapped_word
        
        # joining comment words
        document = " ".join(comment)
        document = document.strip()
                    
        processed_corpus.append(document)
        
    return processed_corpus

# Functions
def get_term_list(path):
    '''
    Function to import term list file
    '''
    word_list = []
    with open(path,"r") as f:
        for line in f:
            word = line.replace("\n","").strip()
            word_list.append(word)
    return word_list

term_badword_list = get_term_list("/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/badwords_list.txt")

In [160]:
import pandas as pd
import re
import unicodedata
import string
import nltk
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import language_tool_python
import pickle  # Importing pickle here
import json
import re
import unicodedata
import string
import nltk
import spacy
import pickle
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
from collections import Counter

# Load the English language model
nlp = spacy.load('en_core_web_sm')
tool = language_tool_python.LanguageTool('en-US')

# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load your DataFrame
# df = pd.read_csv('your_dataset.csv')
tqdm.pandas()  # Initialize tqdm for Pandas integration


label_counts = Counter(df['Cyberbullying'])
print("Label counts before remove duplicates", label_counts)
def preprocess_text(df):
    """Preprocess text data in a DataFrame."""
    
    def _get_ner(x):
        return " ".join([ent.label_ for ent in nlp(x).ents])

    def _get_pos_tag(x):
        return " ".join([token.pos_ for token in nlp(x)])

    def _remove_urls(x):

        return re.sub(r"\b(?:http|https|ftp|ssh)://[^\s]*", '', x)

    def _remove_mention(x):
        return re.sub(r"@\w+", '', x)

    def _remove_emails(x):
        return re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", "", x)

    def _remove_space_single_chars(x):
        temp = re.sub(r'(?i)(?<=\b[a-z]) (?=[a-z]\b)', '', x)
        return temp
    
    def normalize_text(text):
        # Handle specific patterns of laughter
        text = re.sub(r'\b(ha)+\b', 'haha', text, flags=re.IGNORECASE)
        text = re.sub(r'\b(lol)+\b', 'lol', text, flags=re.IGNORECASE)
        text = re.sub(r'\b(lmao)+\b', 'lmao', text, flags=re.IGNORECASE)

        # Reduce elongated sequences of characters
        pattern = re.compile(r"(.)\1{2,}")
        text = pattern.sub(lambda match: match.group(1), text)
        text = re.sub(r'(.)\1{2,}', r'\1', text)
        text = re.sub(r'(.)\1{2,}',r'\1',text)   #any characters, numbers, symbols
        text = re.sub(r'(..)\1{2,}', r'\1', text)  
        text = re.sub(r'(...)\1{2,}', r'\1', text)
        text = re.sub(r'(....)\1{2,}', r'\1', text)  

        return text
    
    def normalize_text_with_original_casing(text):
        # Store original casing of words
        original_casing_mapping = {}
        
        # Find unique words and store their original casing
        for word in set(text.split()):
            original_casing_mapping[word.lower()] = word
        
        # Normalize text by converting to lowercase and reducing elongated words
        normalized_text = normalize_text(text.lower())
        
        # Restore original casing using the mapping
        restored_text = " ".join(original_casing_mapping.get(word, word) for word in normalized_text.split())
        
        return restored_text

    # Define the function to replace emoticons with descriptions
    def replace_emoticons_with_descriptions(text):
    # Load Emoji Dictionary
        emoji_dict_path = '/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/Emoji_Dict.p'
        with open(emoji_dict_path, 'rb') as file:
            emoji_dict = pickle.load(file)

        # Load the emoticon dictionary from the JSON file
        emoticon_dict_path = '/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/emoticon_dict.json'
        with open(emoticon_dict_path, 'r') as file:
            emoticon_dict = json.load(file)
        
        # Define unwanted characters explicitly
        unwanted_chars = "[£♛™→✔♡†☯♫✌®تح♕★ツ☠♚©♥█║▌│☁☀ღ◄ ▲ ► ▼ ◄ ▲ ► ▼▼ ◄ ▲ ► ▼﻿ ◄ ▲ ► ▼ ◄ ▲ ► ▼ ◄ ▲ ► ▼ ◄﻿ ▲ ▼ ◄ ▲ ► ▼ ◄ ▲ ► ▼ ◄▼﻿ ◄ ▲ ►… — … — ¯¯ … ¯ — … ¯ … ¯ ¯ ¯ … – ¯¯ …… ¯¯ ¯ … ¯ ¯¯ ……¯¯ … ¯ ¯ — … ¯¯– – … ¯ ¯ … ¯ ¯¯ … ¯¯ – … – ¯¯ ¯ — — ¯ ¯¯ – … – ¯¯ — — ¯ … ¯ ¯¯ – ¯¯ … – … —– ¯ …… ¯¯ … ¯ — ¯¯ … … ¯]"
        
        # Remove unwanted characters
        text = re.sub(unwanted_chars, " ", text)

        # Replace emoticons with descriptions
        for emoticon, description in emoticon_dict.items():
            text = text.replace(emoticon, description)

        return text
    
    add_emoticon = {'-.-': 'shame',
      '-_-': 'squiting',
      '^.^': 'happy',
      ':0': 'surprise',
      '^-^': 'happy',
      ':33': 'happy face smiley',
      '^__^': 'happy',
      '-____-': 'shame',
      'o_o': 'confused',
      'O_O': 'confused',
      'x3': 'Cute',
      'T T': 'Cry'
      }

    EMOTICONS_EMO.update(add_emoticon)

    pattern_emoticon = u'|'.join(k.replace('|','\\|') for k in EMOTICONS_EMO)
    pattern_emoticon = pattern_emoticon.replace('\\','\\\\')
    pattern_emoticon = pattern_emoticon.replace('(','\\(')
    pattern_emoticon = pattern_emoticon.replace(')','\\)')
    pattern_emoticon = pattern_emoticon.replace('[','\\[')
    pattern_emoticon = pattern_emoticon.replace(']','\\]')
    pattern_emoticon = pattern_emoticon.replace('*','\\*')
    pattern_emoticon = pattern_emoticon.replace('+','\\+')
    pattern_emoticon = pattern_emoticon.replace('^','\\^')
    pattern_emoticon = pattern_emoticon.replace('·','\\·')
    pattern_emoticon = pattern_emoticon.replace('\{','\\{')
    pattern_emoticon = pattern_emoticon.replace('\}','\\}')
    pattern_emoticon = pattern_emoticon.replace('<','\\>')
    pattern_emoticon = pattern_emoticon.replace('>','\\>')
    pattern_emoticon = pattern_emoticon.replace('?','\\?')

    # Convert emoticons into word
    def _convert_emoticons(x):
        for emot in EMOTICONS_EMO:
            x = x.replace(emot, "_".join(EMOTICONS_EMO[emot].replace(",","").replace(":","").split()))
        return x

    # Count emoji
    pattern_emoji = u'|'.join(k.replace('|','\\|') for k in UNICODE_EMOJI)
    pattern_emoji = pattern_emoji.replace('\\','\\\\')
    pattern_emoji = pattern_emoji.replace('(','\\(')
    pattern_emoji = pattern_emoji.replace(')','\\)')
    pattern_emoji = pattern_emoji.replace('[','\\[')
    pattern_emoji = pattern_emoji.replace(']','\\]')
    pattern_emoji = pattern_emoji.replace('*','\\*')
    pattern_emoji = pattern_emoji.replace('+','\\+')
    pattern_emoji = pattern_emoji.replace('^','\\^')
    pattern_emoji = pattern_emoji.replace('·','\\·')
    pattern_emoji = pattern_emoji.replace('\{','\\{·')
    pattern_emoji = pattern_emoji.replace('\}','\\}·')


    # Convert emoji into word
    def _convert_emojis(x):
        for emot in UNICODE_EMOJI:
            x = x.replace(emot, "_".join(UNICODE_EMOJI[emot].replace(",","").replace(":","").split()))
        return x

    def remove_accented_chars(x):
        x = unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        return x

    def _slang_resolution(x):
        slang_path = '/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/SLANG_SOCIAL.pkl'
        with open(slang_path, 'rb') as fp:
            slang_path = pickle.load(fp)
        clean_text = []
        for text in x.split():
            if text in list(slang_path.keys()):
                for key in slang_path:
                    value = slang_path[key]
                    if text == key:
                        clean_text.append(text.replace(key,value))
                    else:
                        continue
            else:
                clean_text.append(text)
        return " ".join(clean_text)

    # Sample function to normalize text with original casing
    def slang_resolution__with_original_casing(text):
        
        # Store original casing of words
        original_casing_mapping = {}
        
        # Find unique words and store their original casing
        for word in set(text.split()):
            original_casing_mapping[word.lower()] = word
        
        # Normalize text by converting to lowercase and reducing elongated words
        normalized_text = _slang_resolution(text.lower())
        
        # Restore original casing using the mapping
        restored_text = " ".join(original_casing_mapping.get(word, word) for word in normalized_text.split())
        
        return restored_text

    # Function to expand contractions in text
    def expand_contractions(text):
        # Define the CONTRACTION_MAP dictionary
        CONTRACTION_MAP = {
        "ain't": "is not",
        "aren't": "are not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he would",
        "he'd've": "he would have",
        "he'll": "he will",
        "he'll've": "he he will have",
        "he's": "he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how is",
        "I'd": "I would",
        "I'd've": "I would have",
        "I'll": "I will",
        "I'll've": "I will have",
        "I'm": "I am",
        "I've": "I have",
        "i'd": "i would",
        "i'd've": "i would have",
        "i'll": "i will",
        "i'll've": "i will have",
        "i'm": "i am",
        "i've": "i have",
        "isn't": "is not",
        "it'd": "it would",
        "it'd've": "it would have",
        "it'll": "it will",
        "it'll've": "it will have",
        "it's": "it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she would",
        "she'd've": "she would have",
        "she'll": "she will",
        "she'll've": "she will have",
        "she's": "she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as",
        "that'd": "that would",
        "that'd've": "that would have",
        "that's": "that is",
        "there'd": "there would",
        "there'd've": "there would have",
        "there's": "there is",
        "they'd": "they would",
        "they'd've": "they would have",
        "they'll": "they will",
        "they'll've": "they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what will",
        "what'll've": "what will have",
        "what're": "what are",
        "what's": "what is",
        "what've": "what have",
        "when's": "when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where is",
        "where've": "where have",
        "who'll": "who will",
        "who'll've": "who will have",
        "who's": "who is",
        "who've": "who have",
        "why's": "why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you would",
        "you'd've": "you would have",
        "you'll": "you will",
        "you'll've": "you will have",
        "you're": "you are",
        "you've": "you have",
        "stfu": "shut the fuck up",
        "wtf": "what the fuck",
        " u ": " you ",
        " ur ": " your ",
        " n ": " and ",
        " dis ": " this ",
        "'d": " would",
        }
        for contraction, expansion in CONTRACTION_MAP.items():
            text = text.replace(contraction, expansion)
        return text

    # Sample function to normalize text with original casing
    def expand_contractions_with_original_casing(text):
        # Store original casing of words
        original_casing_mapping = {}
        
        # Find unique words and store their original casing
        for word in set(text.split()):
            original_casing_mapping[word.lower()] = word
        
        # Normalize text by converting to lowercase and expanding contractions
        normalized_text = expand_contractions(text.lower())
        
        # Restore original casing using the mapping
        restored_text = " ".join(original_casing_mapping.get(word, word) for word in normalized_text.split())
        
        return restored_text

    def _remove_numeric(x):
        return ''.join([i for i in x if not i.isdigit()])

    def _remove_special_chars(x):
        punct = string.punctuation + "¶“”‘’" 
        for p in punct:
            x = x.replace(p, " ")
        return x

    def lemmatize_word(text):
        lemmatizer = WordNetLemmatizer()
        lemmas = [lemmatizer.lemmatize(word) for word in text]
        return lemmas
    
    # Apply preprocessing steps
    print('Text Preprocessing: Developing NER tag count')
    df['ner_tags'] = df['text'].progress_apply(_get_ner)
    print('Text Preprocessing: Developing POS tag count')
    df['pos_tags'] = df['text'].progress_apply(_get_pos_tag)
    print('Text Preprocessing: Remove urls, user mention, emails')
    df['text_check'] = df['text'].progress_apply(lambda x: _remove_urls(x))
    df['text_check'] = df['text_check'].progress_apply(lambda x: _remove_mention(x))
    df['text_check'] = df['text_check'].progress_apply(lambda x: _remove_emails(x))
    print('Text Preprocessing: Remove single characters')
    df['text_check'] = df['text_check'].progress_apply(_remove_space_single_chars)
    print('Text Preprocessing: Reduce elongated characters')
    df['text_check'] = df['text_check'].progress_apply(lambda x: normalize_text_with_original_casing(x))
    print('Text Preprocessing: Reduce accented characters')
    df['text_check'] = df['text_check'].progress_apply(remove_accented_chars)
    print('Text Preprocessing: Expand contraction')
    df['text_check'] = df['text_check'].progress_apply(expand_contractions_with_original_casing)
    print('Text Preprocessing: Correct abbreviation or slang')
    df['text_check'] = df['text_check'].progress_apply(lambda x: slang_resolution__with_original_casing(x))
    print('Text Preprocessing: Normalize emoticons')
    df['text_check'] = df['text_check'].progress_apply(lambda x: _convert_emojis(x))
    df['text_check'] = df['text_check'].progress_apply(lambda x: _convert_emoticons(x))
    df['text_check'] = df['text_check'].progress_apply(replace_emoticons_with_descriptions)
    print('Text Preprocessing: Lowercase')
    df['text_check'] = df['text_check'].str.lower()
    print('Text Preprocessing: Replace obfuscated bad term')
    # unique words in vocab 
    unique_words = get_vocab(corpus= df['text_check'])    
     # creating mapping dict 
    mapping_dict = create_profane_mapping(profane_words=term_badword_list,vocabulary=unique_words)
    df['text_check'] = replace_words(corpus=df['text_check'], mapping_dict=mapping_dict)
    print('Text Preprocessing: Correct spelling')
    df['text_check'] = df['text_check'].progress_apply(lambda x: tool.correct(x))
    print('Text Preprocessing: Remove numeric character')
    df['text_check'] = df['text_check'].progress_apply(_remove_numeric)
    print('Text Preprocessing: Remove punctuations')
    df['text_check'] = df['text_check'].progress_apply(lambda x: _remove_special_chars(x))
    print('Text Preprocessing: Remove multiple spaces')
    df['text_check'] = df['text_check'].progress_apply(lambda x: ' '.join(x.split()))
    print('Text Preprocessing: Tokenisation')
    df["tokenize_text"] = df.progress_apply(lambda row: nltk.word_tokenize(row['text_check'].lower()), axis=1)
    print('Text Preprocessing: Lemmatization')
    df["lemmatized_text"] = df["tokenize_text"].progress_apply(lemmatize_word)
    df['clean_text'] = df['lemmatized_text'].progress_apply(lambda tokens: ' '.join(tokens))
    
    # Remove empty texts
    df = df[~df['clean_text'].isna()]
    df = df[df['clean_text'] != '']
    df = df.reset_index(drop=True)
    
    print('Done')

    return df

Label counts before remove duplicates Counter({0: 118172, 1: 5376})


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tszeyenthen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tszeyenthen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/tszeyenthen/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [175]:
import re

# Extract numeric part from the 'file_name' and create a new column for sorting
df['file_number'] = df['file_name'].apply(lambda x: int(re.search(r'\d+', x).group()))

# Sort the data by the extracted number and 'sentence_id'
sorted_data = df.sort_values(by=['file_number', 'sentence_id'])

# Drop the temporary sorting column
sorted_data.drop('file_number', axis=1, inplace=True)

# Save the newly sorted data to a new CSV file
sorted_numeric_file_path = '/Users/tszeyenthen/Python Study/jupyter notebbok/Cyberbullying/fyp/amica-cyberbullying-distribute/askfm-cyberbullying-data/sorted_numeric_concatenated_results.csv'
sorted_data.to_csv(sorted_numeric_file_path, index=False)

# Show the first few rows to verify sorting
sorted_data.head(), sorted_numeric_file_path

(                           file_name sentence_id  \
 69476  Askfm_conversation_1_main.xml         s.0   
 69477  Askfm_conversation_1_main.xml         s.1   
 69483  Askfm_conversation_1_main.xml        s.10   
 69484  Askfm_conversation_1_main.xml        s.12   
 69485  Askfm_conversation_1_main.xml        s.13   
 
                                                     text  Cyberbullying  \
 69476                      ¶ Have you ever been in love?              0   
 69477                                   ¶ I am in love..              0   
 69483                ¶ My boyfriend. Who are you dating?              0   
 69484  ¶ Well your the true bitch no it not a joke wh...              1   
 69485  ¶ Are you really doing this right now? Please ...              1   
 
            role  harmful_score words0                                words1  \
 69476       NaN            NaN    NaN                                   NaN   
 69477       NaN            NaN    NaN                        

In [294]:
import pandas as pd
from langdetect import detect, LangDetectException

# Function to detect English text
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:  # Handle exception if text is too short or detection fails
        return False
    
# Display the DataFrame before removal for comparison
print("Before removal:")
print(sorted_data.shape)

# Step 2: Filter out rows where 'label1' is 'Other_Language'
sorted_data_files = sorted_data[sorted_data['label1'] != 'Other_language']

# Display the DataFrame after removal to verify the changes
print("After removal:")
print(sorted_data_files.shape)
# sorted_data_files.to_csv('en.csv')

preprocess_text(sorted_data_files)
# after processing
print("After processing:")
print(sorted_data_files.shape)
sorted_data_files.to_csv('en_after_processing.csv')

sorted_data_files_clean = sorted_data_files[['text', 'Cyberbullying']]

# # Rename the columns to 'text' and 'label'
sorted_data_files_clean.columns = ['text', 'label']
sorted_data_files_clean.columns = ['text', 'label']
sorted_data_files_clean = sorted_data_files.dropna(subset=['text'])
sorted_data_files_clean  = sorted_data_files_clean [sorted_data_files_clean ['text'] != '']
sorted_data_files_clean  = sorted_data_files_clean [~sorted_data_files_clean ['text'].isna()]
sorted_data_files_clean  = sorted_data_files_clean .reset_index(drop=True)
print("After processing:")
print(sorted_data_files_clean.shape)
sorted_data_files_clean.to_csv('en_cleaned.csv')

Before removal:
(123548, 64)
After removal:
(114782, 64)
Text Preprocessing: Developing NER tag count


100%|██████████| 114782/114782 [06:15<00:00, 305.83it/s]
/var/folders/h2/q_r6y6bd5f5c9hvfsfnt31xr0000gn/T/ipykernel_10037/4101585095.py:386: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ner_tags'] = df['text'].progress_apply(_get_ner)


Text Preprocessing: Developing POS tag count


100%|██████████| 114782/114782 [06:04<00:00, 315.07it/s]
/var/folders/h2/q_r6y6bd5f5c9hvfsfnt31xr0000gn/T/ipykernel_10037/4101585095.py:388: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pos_tags'] = df['text'].progress_apply(_get_pos_tag)


Text Preprocessing: Remove urls, user mention, emails


...

Text Preprocessing: Tokenisation


100%|██████████| 114782/114782 [00:08<00:00, 13317.11it/s]
/var/folders/h2/q_r6y6bd5f5c9hvfsfnt31xr0000gn/T/ipykernel_10037/4101585095.py:424: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tokenize_text"] = df.progress_apply(lambda row: nltk.word_tokenize(row['text_check'].lower()), axis=1)


Text Preprocessing: Lemmatization


100%|██████████| 114782/114782 [00:01<00:00, 66998.84it/s]
/var/folders/h2/q_r6y6bd5f5c9hvfsfnt31xr0000gn/T/ipykernel_10037/4101585095.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["lemmatized_text"] = df["tokenize_text"].progress_apply(lemmatize_word)
100%|██████████| 114782/114782 [00:00<00:00, 2141661.89it/s]
/var/folders/h2/q_r6y6bd5f5c9hvfsfnt31xr0000gn/T/ipykernel_10037/4101585095.py:427: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['clean_text'] = df['lemmatized_text'].progress_apply(l

Done
After processing:
(114782, 70)
After processing:
(114782, 70)


In [299]:
sorted_data_files_clean = sorted_data_files[['clean_text', 'Cyberbullying']]

# Rename the columns to 'text' and 'label'
sorted_data_files_clean.columns = ['text', 'label']
sorted_data_files_clean = sorted_data_files_clean.dropna(subset=['text'])
sorted_data_files_clean  = sorted_data_files_clean [sorted_data_files_clean ['text'] != '']
sorted_data_files_clean  = sorted_data_files_clean [~sorted_data_files_clean ['text'].isna()]
# sorted_data_files_clean  = sorted_data_files_clean .reset_index(drop=True)
print("After processing:")
print(sorted_data_files_clean.shape)
sorted_data_files_clean.to_csv('en_cleaned.csv')

After processing:
(112481, 2)
